In [4]:
import pandas as pd

In [6]:
my_hppr = pd.read_csv('my_hppr_rank.csv')
fp_ecr = pd.read_csv('draft_in_progress.csv')

In [138]:
fp_ecr.head(36)

,RK,TIERS,PLAYER NAME,TEAM,POS,BYE WEEK,SOS SEASON,ECR VS. ADP
0,1,1,Christian McCaffrey,SF,RB1,9,2 out of 5 stars,0
1,2,1,CeeDee Lamb,DAL,WR1,7,3 out of 5 stars,0
2,3,1,Tyreek Hill,MIA,WR2,6,2 out of 5 stars,0
3,4,1,Ja'Marr Chase,CIN,WR3,12,2 out of 5 stars,+2
4,5,1,Bijan Robinson,ATL,RB2,12,4 out of 5 stars,+2
5,6,1,Justin Jefferson,MIN,WR4,6,3 out of 5 stars,-1
6,7,1,Amon-Ra St. Brown,DET,WR5,5,4 out of 5 stars,+1
7,8,1,Breece Hall,NYJ,RB3,12,2 out of 5 stars,-4
8,9,2,A.J. Brown,PHI,WR6,5,2 out of 5 stars,0
9,10,2,Garrett Wilson,NYJ,WR7,12,4 out of 5 stars,+4


In [99]:
def snake_draft(curr_pick, league_size):
    round = (curr_pick - 1) // league_size
    if round%2 == 0:
        position = curr_pick - (league_size * round)
        next_pick = curr_pick + ((league_size - position) * 2) + 1
    else:
        position = (league_size * (round+ 1)) - curr_pick + 1
        next_pick = curr_pick + ((position) * 2) - 1
    return next_pick

In [100]:
snake_draft(163, 12)

174

In [124]:
def get_best_by_pos(my_rank):
    return my_rank[my_rank.groupby('Position')['HPPR'].transform('max') == my_rank['HPPR']]

In [125]:
get_best_by_pos(pd.read_csv('my_hppr_rank.csv'))

,Player Name,Position,HPPR,HPPR per Game
0,Anthony Richardson,QB,405.1,23.831392
3,Christian McCaffrey,RB,362.7,21.338233
4,CeeDee Lamb,WR,360.7,21.219620
48,Travis Kelce,TE,240.5,14.147432


In [153]:
def pos_opp_cost(curr_best, next_best):
    return_dict = {}
    for position in ['QB', 'WR', 'TE', 'RB']:
        curr_best_row = curr_best[curr_best['Position'] == position]
        curr_best_hpprpg = curr_best_row['HPPR per Game'].values[0]
        next_best_row = next_best[next_best['Position'] == position]
        next_best_hpprpg = next_best_row['HPPR per Game'].values[0]
        return_dict[position] = {
            'curr_best_name' : curr_best_row['Player Name'].values[0],
            'curr_best_hppr' : curr_best_row['HPPR per Game'].values[0],
            'next_best_name' : next_best_row['Player Name'].values[0],
            'next_best_hppr' : next_best_row['HPPR per Game'].values[0],
            'diff' : curr_best_row['HPPR per Game'].values[0] - next_best_row['HPPR per Game'].values[0]
        }
    return return_dict

In [154]:
# who are the best players now, and who are the best players likely remaining after x rounds.

def best_options(curr_pick, league_size, my_rank, ecr):
    current_best = get_best_by_pos(my_rank)
    next_round = snake_draft(curr_pick, league_size)
    rounds_to_go = next_round - curr_pick
    get_ecr_picks = ecr['PLAYER NAME'].head(rounds_to_go)
    my_rank = my_rank[my_rank["Player Name"].str.contains('|'.join(list(get_ecr_picks))) == False]
    next_best = get_best_by_pos(my_rank)
    return pos_opp_cost(current_best, next_best)


In [155]:
best_options(2, 12, my_hppr, fp_ecr)

{'QB': {'curr_best_name': 'Anthony Richardson',
  'curr_best_hppr': np.float64(23.83139194),
  'next_best_name': 'Anthony Richardson',
  'next_best_hppr': np.float64(23.83139194),
  'diff': np.float64(0.0)},
 'WR': {'curr_best_name': 'CeeDee Lamb',
  'curr_best_hppr': np.float64(21.2196201),
  'next_best_name': 'Michael Pittman Jr.',
  'next_best_hppr': np.float64(14.20684322),
  'diff': np.float64(7.0127768800000005)},
 'TE': {'curr_best_name': 'Travis Kelce',
  'curr_best_hppr': np.float64(14.1474316),
  'next_best_name': 'Travis Kelce',
  'next_best_hppr': np.float64(14.1474316),
  'diff': np.float64(0.0)},
 'RB': {'curr_best_name': 'Christian McCaffrey',
  'curr_best_hppr': np.float64(21.33823301),
  'next_best_name': 'Devone Achane',
  'next_best_hppr': np.float64(19.4133609),
  'diff': np.float64(1.924872109999999)}}